#### ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metric
- 문서를 요약했는데, 어떻게 품질을 측정하나?
- 겹침정도를 측정 (ex. 문장내에서 핵심적인 단어가 요약문제 없다? 그럼 요약이 잘 안된거지 않을까?)
- ROUGE-1 : 1개 단어단위로 겹침정도를 판단
    - 정답 : 고양이가 생선을 먹었다
    - 생성 : 고양이가 물고기를 먹었다
    - 겹침 : 고양이가, 먹었다
    - =====> 2/4 = 0.5
- ROUGE-2 : 2개 단어단위로 겹침정도를 판단 : 순서도 고려 대상이 됨
- ROUGE-L : 가장 긴 공통 부분수열로 겹침정도 판단

- 단점: 의미는 같지만, 다른표현을 쓰면 점수가 낮음.
    - 자동차 vs 차량 : 겹침없음으로 판단함.

In [1]:
# 토크나이저

In [2]:
# 배치처리, 데이터 콜레이터 (배치를 만들때 길이를 맞춰주는 작업)
# 효율적인 데이터 처리방법을 알아보자

- 개별 처리:
    - [문장1] -> 모델 -> [출력1]
    - [문장2] -> 모델 -> [출력2]

- 배치 처리:   [문장1, 문장2, ...] -> 모델 -> [출력1, 출력2, ...]

- DataCollator:  토큰화 + 패딩 + 라벨 시프트 + tensor 변환 -> 모델

In [3]:
import torch, time
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
    # DataCollatorForSeq2Seq :
    # seq2seq(Encoder-Decoder) 모델 학습용 데이터 배치 전처리를 자동화해주는 유틸
    # 배치 단위에서 모델 입력과 라벨을 자동으로 맞춰주는 도우미
    # seq2seq 학습시, 배치단위로 "패딩-정렬-라벨시프트" 등을 자동처리하는 데이터 정렬도구
        # 배치생성 - 길이가 다른 문장들을 동일길이로 패딩하는 것을 일컫음
        # 라벨 시프트 - 디코더 입력과 라벨이 한칸씩 밀리도록 자동 변환 (teacher forcing에 필요한 작업)
        # 배치단위 tensor 변환 - list of dict 형태를 torch.Tensor로 변환
        # DataLoader 안에서 사용

    # AutoModelForSeq2SeqLM :
    # 입력을 받아서 다른 텍스트를 생성하는 seq2seq 모델을 자동로드
        # ---> Seq2Seq이므로 Encoder-Decoder 모델을 자동으로
        # ---> 이 라이브러리 안에는 T5, BART, MarianMT 모델이 있음
        # ---> 번역/요약/Q&A/문장변환에 최적화 (입력들어오면 출력하는 형태에 최적화)

# 모델 : t5-small을 갖는 생성형 모델
MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

texts = [
    "summarize: The cat sat on the mat.",
    "summarize: Python is a popular programming language.",
    "summarize: Machine learning is a subset of artificial intelligence.",
    "summarize: The weather is nice today.",
    "summarize: I love reading books in my free time.",
    "summarize: Coffee is one of the most popular beverages worldwide.",
    "summarize: Regular exercise is important for health.",
    "summarize: The Internet has changed how we communicate.",
]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

c:\Users\playdata2\miniconda3\envs\P10\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\playdata2\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [4]:
# 개별처리 vs 배치처리


### 개별처리
# 특징: 문장 하나씩 모델에 입력 -> 출력 -> 다음문장 처리
# 장점: 단순, 직관적
# 단점: 속도 느림, GPU활용 비효율

start_time = time.time()
result_individual = []
with torch.no_grad():
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True).to(device)
        # print(inputs) # inputs 안에 들어있는거 확인 : input_ids, attention_mask 2개 들어있음
        outputs = model.generate(**inputs, max_length=30)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        result_individual.append(summary)
time_indivisual = time.time() - start_time
print(f"개별처리 소요시간: {time_indivisual:.2f}초")
print(f"개별처리 문장당 소요: {time_indivisual/ len(texts): .4f}초")

개별처리 소요시간: 3.88초
개별처리 문장당 소요:  0.4847초


In [5]:
### 배치처리
# 특징: 여러문장을 모아서 한번에 모델에 입력
# 장점: GPU 병렬 처리로 속도 빠름, 효율적
# 단점: 문장 길이가 다르면 패딩 필요

start_time = time.time()
result_batch = []
batch_inputs = tokenizer(
    texts,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
).to(device)
outputs = model.generate(**batch_inputs, max_length=30)
for output in outputs:
    summary = tokenizer.decode(output, skip_sepcial_tokens=True)
    result_batch.append(summary)
time_batch = time.time() - start_time
print(f"배치처리 소요시간: {time_batch:.2f}초")
print(f"배치처리 문장당 소요: {time_batch/ len(texts): .4f}초")

배치처리 소요시간: 0.58초
배치처리 문장당 소요:  0.0727초


In [8]:
# DataCollatorForSeq2Seq  seq2se1 학습용 배치 생성
# 특징 : 배치 생성, 패딩, 라벨 시프트 등 seq2seq 학습 전처리 자동화
# 장점:
    # 여러 문장을 한 번에 배치로 만들어 줌
    # 길이가 서로 다른 문장 자동 패딩
    # 라벨 시프트(teacher forcing) 자동 처리
    # PyTorch Tensor로 변환 → 바로 모델 입력 가능


start_time = time.time()
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
tokenized = []
for text in texts:
    encoded = tokenizer(text,truncation=True)
    tokenized.append(encoded)
batch = data_collator(tokenized)
time_indivisual = time.time() - start_time
print(batch)
print(f"소요시간: {time_indivisual:.2f}초")
print(f"문장당: {time_indivisual/ len(texts): .4f}초")

{'input_ids': tensor([[21603,    10,    37,  1712,     3,     7,   144,    30,     8,  6928,
             5,     1,     0,     0],
        [21603,    10, 20737,    19,     3,     9,  1012,  6020,  1612,     5,
             1,     0,     0,     0],
        [21603,    10,  5879,  1036,    19,     3,     9,   769,  2244,    13,
          7353,  6123,     5,     1],
        [21603,    10,    37,  1969,    19,  1245,   469,     5,     1,     0,
             0,     0,     0,     0],
        [21603,    10,    27,   333,  1183,  1335,    16,    82,   339,    97,
             5,     1,     0,     0],
        [21603,    10, 10429,    19,    80,    13,     8,   167,  1012, 18928,
          4388,     5,     1,     0],
        [21603,    10, 17116,  2510,    19,   359,    21,   533,     5,     1,
             0,     0,     0,     0],
        [21603,    10,    37,  1284,    65,  2130,   149,    62,  4521,     5,
             1,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 